In [ ]:
#!pip install albumentations
import albumentations

In [ ]:
import torch
from albumentations import ( Compose, OneOf, Normalize, Resize, RandomResizedCrop, RandomCrop, HorizontalFlip, VerticalFlip, 
    RandomBrightness, RandomContrast, RandomBrightnessContrast, Rotate, ShiftScaleRotate, Cutout, IAAAdditiveGaussianNoise, Transpose, ToGray )
from albumentations.pytorch import ToTensorV2
from albumentations import ImageOnlyTransform

import matplotlib.pyplot as plt

seed = 42

import pandas as pd
import os
import cv2
from torch.utils.data import Dataset,DataLoader
from tqdm import tqdm



class Ranzcr_jpg_train_dataset(Dataset):    
	
    def __init__(self, files_folder_path, df, num_channels , transfroms = None ):
		self.files_folder_path = files_folder_path
		self.df = df
		self.transforms = transfroms
		self.num_channels = num_channels

	def __len__(self):
		return len(self.df)

	def __getitem__(self, idx):

		image_id = self.df.StudyInstanceUID.values[idx]
		if self.num_channels == 1:
			image = cv2.imread(os.path.join(self.files_folder_path, image_id + ".jpg" ), 0)
        
		else:
			image = cv2.imread(os.path.join(self.files_folder_path, image_id + ".jpg" ))
			image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = image #*(1/255)
        
		if self.transforms:
			image = self.transforms(image=image)['image']
		labels = self.df[self.df.StudyInstanceUID == image_id].values.tolist()[0][1:-1]
		labels = torch.tensor(labels,dtype= torch.float32) #.view(1,-1)

		return image, labels

In [ ]:
class Ranzcr_jpg_train_dataset(Dataset):    
	def __init__(self, files_folder_path, df, num_channels , transfroms = None ):
		self.files_folder_path = files_folder_path
		self.df = df
		self.transforms = transfroms
		self.num_channels = num_channels

	def __len__(self):
		return len(self.df)

	def __getitem__(self, idx):

		image_id = self.df.StudyInstanceUID.values[idx]
		image = cv2.imread(os.path.join(self.files_folder_path, image_id + ".jpg" ))
		image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
		image = image*(1/255)
        
		if self.transforms:
			image = self.transforms(image=image)['image']
		labels = self.df[self.df.StudyInstanceUID == image_id].values.tolist()[0][1:-1]
		labels = torch.tensor(labels,dtype= torch.float32) #.view(1,-1)

		return image, labels

In [ ]:
train_path = '../input/ranzcr-clip-catheter-line-classification/train'
train_files = os.listdir(train_path)

train_df = pd.read_csv('../input/ranzcr-clip-catheter-line-classification/train.csv')

train = train_df.reset_index(drop=True) # reset index on both dataframes

print(train.shape)

num_channel = 3

img_size = 255

# Try different Mean and standard deviations

In [ ]:
mean_list = [[ 0.4914168 , 0.4914168 , 0.4914168] , [0.485,0.456,0.406] , [0.9,0.9,0.9] , 0.496 ]
std_list = [[0.407278, 0.407278 , 0.407278] , [0.229,0.224,0.225] , [0.9,0.9,0.9] , 0.407278]

mean = mean_list[0]
std=  std_list[0]

norm_255 = False #False


#Not needed. Instead pass value of max_pixel value.

if norm_255 == True:
	for x in range(0,3):
		mean[x] = mean[x]*255
		std[x] = std[x]*255


print(mean , std)



Image = Image/255

In [ ]:
recale_image_by = 1
max_pixel_value = 255 


class Ranzcr_jpg_train_dataset(Dataset):    
	def __init__(self, files_folder_path, df, num_channels , transfroms = None ):
		self.files_folder_path = files_folder_path
		self.df = df
		self.transforms = transfroms
		self.num_channels = num_channels

	def __len__(self):
		return len(self.df)

	def __getitem__(self, idx):

		image_id = self.df.StudyInstanceUID.values[idx]
		image = cv2.imread(os.path.join(self.files_folder_path, image_id + ".jpg" ))
		image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
		image = image*(1/recale_image_by)
        
		if self.transforms:
			image = self.transforms(image=image)['image']
		labels = self.df[self.df.StudyInstanceUID == image_id].values.tolist()[0][1:-1]
		labels = torch.tensor(labels,dtype= torch.float32) #.view(1,-1)

		return image, labels
    
    
def Show_Xrays(augmentation):
    num_channels = 3
    trainset = Ranzcr_jpg_train_dataset(train_path,  train, num_channels , augmentation )
    trainloader = DataLoader(trainset, batch_size = 1 , num_workers = 3 , shuffle = False)

    fig = plt.figure()
    fig.set_size_inches(8, 8)
    #fig.savefig('test2png.png', dpi=100)

    for batch_i, (data, target) in tqdm(enumerate(trainloader)):
        #print(data.shape)
        if batch_i == 1:
            break
        for i in range(data.shape[0]):
            ax = plt.subplot(1,1, i+1)
            plt.tight_layout()
            ax.axis('off')
            plt.imshow(data[i])
            print(data[i])

for i in range(0,4):
    mean = mean_list[i]
    std = std_list[i]
    print(mean , std)
    train_augs = albumentations.Compose([ albumentations.Resize(height=img_size, width=img_size, p=1.0), 
                                         albumentations.Normalize(mean= mean ,std= std , max_pixel_value=max_pixel_value),])

    Show_Xrays(train_augs)

# Reducing image pixels by a factor of 255 first

Not needed

mean and std are already scaled in albumentations , so we dont need to multiply mean/std by 255. [code_here](https://github.com/albumentations-team/albumentations/blob/ff83de8a51184bca97be3a9fc6905c56b44c494a/albumentations/augmentations/functional.py#L129)

In [ ]:
recale_image_by = 255
max_pixel_value = 1 


class Ranzcr_jpg_train_dataset(Dataset):    
	def __init__(self, files_folder_path, df, num_channels , transfroms = None ):
		self.files_folder_path = files_folder_path
		self.df = df
		self.transforms = transfroms
		self.num_channels = num_channels

	def __len__(self):
		return len(self.df)

	def __getitem__(self, idx):

		image_id = self.df.StudyInstanceUID.values[idx]
		image = cv2.imread(os.path.join(self.files_folder_path, image_id + ".jpg" ))
		image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
		image = image*(1/recale_image_by)
        
		if self.transforms:
			image = self.transforms(image=image)['image']
		labels = self.df[self.df.StudyInstanceUID == image_id].values.tolist()[0][1:-1]
		labels = torch.tensor(labels,dtype= torch.float32) #.view(1,-1)

		return image, labels
    
    
def Show_Xrays(augmentation):
    num_channels = 3
    trainset = Ranzcr_jpg_train_dataset(train_path,  train, num_channels , augmentation )
    trainloader = DataLoader(trainset, batch_size = 1 , num_workers = 3 , shuffle = False)

    fig = plt.figure()
    fig.set_size_inches(8, 8)
    #fig.savefig('test2png.png', dpi=100)

    for batch_i, (data, target) in tqdm(enumerate(trainloader)):
        #print(data.shape)
        if batch_i == 1:
            break
        for i in range(data.shape[0]):
            ax = plt.subplot(1,1, i+1)
            plt.tight_layout()
            ax.axis('off')
            plt.imshow(data[i])
            print(data[i])

for i in range(0,4):
    mean = mean_list[i]
    std = std_list[i]
    print(mean , std)
    train_augs = albumentations.Compose([ albumentations.Resize(height=img_size, width=img_size, p=1.0), 
                                         albumentations.Normalize(mean= mean ,std= std , max_pixel_value=max_pixel_value),])

    Show_Xrays(train_augs)

# Baseline

In [ ]:
class Ranzcr_jpg_train_dataset(Dataset):    
	def __init__(self, files_folder_path, df, num_channels , transfroms = None ):
		self.files_folder_path = files_folder_path
		self.df = df
		self.transforms = transfroms
		self.num_channels = num_channels

	def __len__(self):
		return len(self.df)

	def __getitem__(self, idx):

		image_id = self.df.StudyInstanceUID.values[idx]
		image = cv2.imread(os.path.join(self.files_folder_path, image_id + ".jpg" ))
		image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
		#image = image*(1/recale_image_by)
        
		if self.transforms:
			image = self.transforms(image=image)['image']
		labels = self.df[self.df.StudyInstanceUID == image_id].values.tolist()[0][1:-1]
		labels = torch.tensor(labels,dtype= torch.float32) #.view(1,-1)

		return image, labels

In [ ]:

def Show_Xrays(augmentation):
    num_channels = 3
    trainset = Ranzcr_jpg_train_dataset(train_path,  train, num_channels , augmentation )
    trainloader = DataLoader(trainset, batch_size = 9 , num_workers = 3 , shuffle = False)

    fig = plt.figure()
    fig.set_size_inches(25, 25)
    #fig.savefig('test2png.png', dpi=100)

    for batch_i, (data, target) in tqdm(enumerate(trainloader)):
        #print(data.shape)
        if batch_i == 1:
            break
        for i in range(data.shape[0]):
            ax = plt.subplot(3,3, i+1)
            plt.tight_layout()
            ax.axis('off')
            plt.imshow(data[i])
            

In [ ]:
train_augs = albumentations.Compose([ albumentations.Resize(height=img_size, width=img_size, p=1.0), 
                                     #albumentations.Normalize(mean= mean ,std= std ,),
                                    ])

Show_Xrays(train_augs)

In [ ]:
train_augs = albumentations.Compose([ albumentations.Resize(height=img_size, width=img_size, p=1.0), 
                                     albumentations.Normalize(mean= mean ,std= std ,p = 1),
                                    ])

Show_Xrays(train_augs)

In [ ]:
train_augs = albumentations.Compose([   albumentations.RandomResizedCrop(img_size, img_size, scale=(0.9, 1), p=1),
                                        albumentations.RandomBrightnessContrast(brightness_limit=(-0.2,0.2), contrast_limit=(-0.2, 0.2), p=0.7),                            
                                    ])
Show_Xrays(train_augs)

In [ ]:
train_augs = albumentations.Compose([   albumentations.RandomResizedCrop(img_size, img_size, scale=(0.9, 1), p=1),
                                        albumentations.RandomBrightnessContrast(brightness_limit=(-0.2,0.2), contrast_limit=(-0.2, 0.2), p=0.7), 
                                        albumentations.CLAHE(clip_limit=(1,10), p= 1),
                                        #albumentations.Normalize(mean= mean ,std= std ,)
                                    ])
Show_Xrays(train_augs)

In [ ]:
train_augs = albumentations.Compose([   albumentations.RandomResizedCrop(img_size, img_size, scale=(0.9, 1), p=1),
                                        albumentations.RandomBrightnessContrast(brightness_limit=(-0.2,0.2), contrast_limit=(-0.2, 0.2), p=0.7), 
                                        albumentations.CLAHE(clip_limit=(1,10), p= 1),
                                        albumentations.Normalize(mean= mean ,std= std ,)
                                    ])
Show_Xrays(train_augs)

# Normalizing data first

In [ ]:
train_augs = albumentations.Compose([   albumentations.Normalize(mean= mean ,std= std ,) , 
                                     albumentations.RandomResizedCrop(img_size, img_size, scale=(0.9, 1), p=1),
                                        albumentations.RandomBrightnessContrast(brightness_limit=(-0.2,0.2), contrast_limit=(-0.2, 0.2), p=0.7), 
                                        #albumentations.CLAHE(clip_limit=(1,10), p= 1)
                                        
                                    ])
Show_Xrays(train_augs)

In [ ]:
ShiftScaleRotate(always_apply=False, p=1, shift_limit_x=(-0.0625, 0.0625),
                 shift_limit_y=(-0.0625, 0.0625), scale_limit=(-0.09999999999999998, 0.10000000000000009),
                 rotate_limit=(-45, 45), interpolation=1, border_mode=4, value=None, mask_value=None)

# Trying differnt combinations of image augmentation

(I hope to find one that magically works!)

In [ ]:
train_augs = albumentations.Compose([albumentations.Resize(img_size, img_size),
                albumentations.RandomResizedCrop(img_size, img_size, scale=(0.9, 1), p=1), 
				albumentations.HueSaturationValue(hue_shift_limit=10, sat_shift_limit=10, val_shift_limit=10, p=1),
				albumentations.RandomBrightnessContrast(brightness_limit=(-0.2,0.2), contrast_limit=(-0.2, 0.2), p=1),
				albumentations.CLAHE(clip_limit=(1,4), p=1),
                #albumentations.imgaug.transforms.IAASharpen(alpha=(0.2, 0.3), lightness=(0.5, 0.7), p=1),
                #albumentations.Cutout(max_h_size=int(img_size * 0.05), max_w_size=int(img_size * 0.05), num_holes=5, p= 0.5),
				#albumentations.Normalize(mean= mean ,  std= std ,) 
               ])

Show_Xrays(train_augs)



In [ ]:
train_augs = albumentations.Compose([albumentations.Resize(img_size, img_size),
                albumentations.RandomResizedCrop(img_size, img_size, scale=(0.9, 1), p=1), 
				#albumentations.ShiftScaleRotate(shift_limit_x=(-0.0125, 0.0125),shift_limit_y=(-0.0125, 0.0125) ,rotate_limit=(-15, 15) , p=1),
				albumentations.HueSaturationValue(hue_shift_limit=10, sat_shift_limit=10, val_shift_limit=10, p=1),
				albumentations.RandomBrightnessContrast(brightness_limit=(-0.2,0.2), contrast_limit=(-0.2, 0.2), p=1),
				albumentations.CLAHE(clip_limit=(1,4), p=1),
                ########albumentations.OpticalDistortion(distort_limit=1.0),
				#####albumentations.ElasticTransform(alpha=3),
                #albumentations.GaussNoise(var_limit=[10, 50], p=1),
                #albumentations.MotionBlur(p=1),
                #albumentations.MedianBlur(p=1),
                #albumentations.augmentations.transforms.ISONoise(color_shift=(0.01, 0.05), intensity=(0.1, 0.5), p=1),
                albumentations.imgaug.transforms.IAASharpen(alpha=(0.2, 0.5), lightness=(0.5, 1.0), p=1),
                albumentations.imgaug.transforms.IAAEmboss(alpha=(0.2, 0.5), strength=(0.2, 0.7), p=1),
                ###albumentations.imgaug.transforms.IAAPerspective (scale=(0.05, 0.1), keep_size=True, p=1),
                albumentations.augmentations.transforms.ToGray(p=1),
                #albumentations.augmentations.transforms.RandomGamma(gamma_limit=(80, 120), eps=None, p=1),
                #albumentations.Cutout(max_h_size=int(img_size * 0.05), max_w_size=int(img_size * 0.05), num_holes=5, p= 0.5),
				#albumentations.Normalize(mean= mean ,  std= std ,) 
               ])

Show_Xrays(train_augs)



In [ ]:
mean = [0.485,0.456,0.406]
std = [0.229,0.224,0.225]

transform = albumentations.Compose([  albumentations.RandomCrop(width=256, height=256),
    albumentations.Normalize(mean= mean ,  std= std ,) 
])

# Read an image with OpenCV and convert it to the RGB colorspace
image = cv2.imread("../input/ranzcr-clip-catheter-line-classification/test/1.2.826.0.1.3680043.8.498.10003659706701445041816900371598078663.jpg")
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Augment an image
transformed = transform(image=image)
transformed_image = transformed["image"]


fig = plt.figure()
fig.set_size_inches(15, 15)

ax = plt.subplot(1,1, 1)
plt.tight_layout()
ax.axis('off')
plt.imshow(image)

# Saving all varints of augmentation to folder

https://albumentations.ai/docs/api_reference/augmentations/transforms/#albumentations.augmentations.transforms.RandomRain

In [ ]:
reference_images = 

In [ ]:
augment_list = [albumentations.Resize(img_size, img_size),
                albumentations.RandomResizedCrop(img_size, img_size, scale=(0.9, 1), p=1), 
				albumentations.ShiftScaleRotate(p=1),
				albumentations.HueSaturationValue(hue_shift_limit=10, sat_shift_limit=10, val_shift_limit=10, p=1),
				albumentations.RandomBrightnessContrast(brightness_limit=(-0.2,0.2), contrast_limit=(-0.2, 0.2), p=1),
				albumentations.CLAHE(clip_limit=(1,4), p=1),
                albumentations.OpticalDistortion(distort_limit=1.0),
				albumentations.ElasticTransform(alpha=3),
                albumentations.GaussNoise(var_limit=[10, 50], p=1),
                albumentations.MotionBlur(p=1),
                albumentations.MedianBlur(p=1),
                albumentations.augmentations.transforms.ISONoise(color_shift=(0.01, 0.05), intensity=(0.1, 0.5), p=1),
                albumentations.imgaug.transforms.IAASharpen(alpha=(0.2, 0.5), lightness=(0.5, 1.0), p=1),
                albumentations.imgaug.transforms.IAAEmboss(alpha=(0.2, 0.5), strength=(0.2, 0.7), p=1),
                albumentations.imgaug.transforms.IAAPerspective (scale=(0.05, 0.1), keep_size=True, p=1),
                albumentations.augmentations.transforms.ToGray(p=1),
                albumentations.augmentations.transforms.RandomGamma(gamma_limit=(80, 120), eps=None, p=1),
                albumentations.Cutout(max_h_size=int(img_size * 0.05), max_w_size=int(img_size * 0.05), num_holes=10, p= 0.5),
				albumentations.Normalize(mean= mean ,  std= std ,) 
               ]

In [ ]:
augment_list = [  albumentations.Resize(img_size, img_size),
                albumentations.RandomResizedCrop(img_size, img_size, scale=(0.9, 1), p=1), 
				  ########albumentations.HorizontalFlip(p=1),
				  albumentations.ShiftScaleRotate(p=1),
				   albumentations.HueSaturationValue(hue_shift_limit=10, sat_shift_limit=10, val_shift_limit=10, p=1),
				   albumentations.RandomBrightnessContrast(brightness_limit=(-0.2,0.2), contrast_limit=(-0.2, 0.2), p=1),
				   albumentations.CLAHE(clip_limit=(1,4), p=1),
                   albumentations.OpticalDistortion(distort_limit=1.0),
				   #albumentations.GridDistortion(num_steps=5, distort_limit=1.),
				   albumentations.ElasticTransform(alpha=3),
                 albumentations.GaussNoise(var_limit=[10, 50], p=1),
                 #albumentations.GaussianBlur(p=1),
                 albumentations.MotionBlur(p=1),
                albumentations.MedianBlur(p=1),
                albumentations.augmentations.transforms.ISONoise(color_shift=(0.01, 0.05), intensity=(0.1, 0.5), p=1),
                #albumentations.imgaug.transforms.IAASuperpixels(p_replace=0.1, n_segments=100, p=1),
                albumentations.imgaug.transforms.IAASharpen(alpha=(0.2, 0.5), lightness=(0.5, 1.0), p=1),
                albumentations.imgaug.transforms.IAAEmboss(alpha=(0.2, 0.5), strength=(0.2, 0.7), p=1),
                albumentations.imgaug.transforms.IAAPerspective (scale=(0.05, 0.1), keep_size=True, p=1),
                ######albumentations.augmentations.domain_adaptation.HistogramMatching (reference_images , blend_ratio=(0.5, 1.0), p=0.5),
                #albumentations.augmentations.transforms.ToSepia(p=1),
                albumentations.augmentations.transforms.ToGray(p=1),
                #albumentations.imgaug.transforms.IAAAdditiveGaussianNoise(loc=0, scale=(2.5500000000000003, 12.75), per_channel=False, p=1),
                
                albumentations.augmentations.transforms.RandomGamma(gamma_limit=(80, 120), eps=None, p=1),
                #albumentations.augmentations.transforms.Solarize(threshold=128, p=1),
                #albumentations.augmentations.transforms.RandomFog(fog_coef_lower=0.3, fog_coef_upper=1, alpha_coef=0.08, p=1),
                #albumentations.augmentations.transforms.RandomRain(slant_lower=-10, slant_upper=10, drop_length=20, drop_width=1, drop_color=(200, 200, 200), blur_value=7, brightness_coefficient=0.7, rain_type=None, p=1),
				#albumentations.Cutout(max_h_size=int(img_size * 0.1), max_w_size=int(img_size * 0.1), num_holes=5, p=1),
				albumentations.Normalize(mean= mean ,  std= std ,) 
               ]

In [ ]:
def Show_save_Xrays(augmentation , name):
    num_channels = 3
    trainset = Ranzcr_jpg_train_dataset(train_path,  train, num_channels , augmentation )
    trainloader = DataLoader(trainset, batch_size = 1 , num_workers = 3 , shuffle = False)

    fig = plt.figure()
    fig.set_size_inches(25, 25)
    #fig.savefig('test2png.png', dpi=100)

    for batch_i, (data, target) in tqdm(enumerate(trainloader)):
        #print(data.shape)
        if batch_i == 1:
            break
        for i in range(data.shape[0]):
            ax = plt.subplot(1 ,1, i+1)
            plt.tight_layout()
            ax.axis('off')
            plt.imshow(data[i])
            fig.savefig( str(name) + '.png', dpi=200)

In [ ]:
Save_to_folder = False


if Save_to_folder == True:
    for i in range(len(augment_list)):
        train_augs = albumentations.Compose([ albumentations.Resize(img_size, img_size) , augment_list[i]])
        Show_save_Xrays(train_augs , augment_list[i])

In [ ]:
!zip -r -q 'Albumentation_1*1.zip'  ./

In [ ]:
!rm ./*.zip

In [ ]:
!rm ./*.png

# Finding your our mean and std for image dataset

In [ ]:
def get_mean_std(loader):
    # var[X] = E[X**2] - E[X]**2
    channels_sum, channels_sqrd_sum, num_batches = 0, 0, 0

    for data, _ in tqdm(loader):
        #channels_sum += torch.mean(data, dim=[0, 2, 3])
        channels_sum += torch.Tensor.float(data ).mean(2, dim=[0, 2, 3])
        
        channels_sqrd_sum += torch.Tensor.float(data ** 2, dim=[0, 2, 3])
        #channels_sqrd_sum += torch.mean(data ** 2, dim=[0, 2, 3])
        num_batches += 1

    mean = channels_sum / num_batches
    std = (channels_sqrd_sum / num_batches - mean ** 2) ** 0.5

    return mean, std

In [ ]:
train_augs = albumentations.Compose([ albumentations.Resize(img_size, img_size),    ToTensorV2()     ])

trainset = Ranzcr_jpg_train_dataset(train_path,  train, num_channels , train_augs )
trainloader = DataLoader(trainset, batch_size = 9 , num_workers = 3 , shuffle = False)

In [ ]:
mean = [125.3113, 125.3113, 125.3113]
std = [103.8559, 103.8559, 103.8559]

In [ ]:
for i in range(0,3):
    a = (((std[i] - mean[i] )**2)/417)**0.5
    print(a)

In [ ]:
for i in range(0,3):
    a = (std[i]*(1/255))
    print(a)

In [ ]:
std = [0.407278, 0.407278 , 0.407278]

In [ ]:
for i in range(0,3):
    a = (mean[i]*(1/255))
    print(a)

In [ ]:
mean = [ 0.4914168 , 0.4914168 , 0.4914168]
std = [0.407278, 0.407278 , 0.407278]

In [ ]:
R_channel = 0
G_channel = 0
B_channel = 0

pathDir = '../input/ranzcr-clip-catheter-line-classification/test/*.jpg'

from glob import glob
img_list = glob(pathDir)
print(len(img_list))

In [ ]:
R_total , G_total ,B_total = 0, 0 , 0

total_pixel = 0
for idx in range(len(img_list)):
    filename = img_list[idx]
    img = plt.imread(filename)

    total_pixel = total_pixel + img.shape[0] * img.shape[1]

    R_total += np.sum((img[:, :, 0] - R_mean) ** 2)
    G_total = G_total + np.sum((img[:, :, 1] - G_mean) ** 2)
    B_total = B_total + np.sum((img[:, :, 2] - B_mean) ** 2)

    
    
R_std = sqrt(R_total / total_count)
G_std = sqrt(G_total / total_count)
B_std = sqrt(B_total / total_count)


traindata = datasets.ImageFolder(data_dir + '/train', transforms.ToTensor())
image_means = torch.stack([t.mean(1).mean(1) for t, c in traindata])
image_means.mean(0)
